In [ ]:
import numpy as np
import matplotlib.pyplot as pl

import matplotlib as mpl

pl.style.use('fivethirtyeight')


#mpl.rc('figure', figsize=(8,6))

#mpl.rc('figure', figsize=(14,12))
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

mpl.rcParams['xtick.bottom']=True
mpl.rcParams['ytick.left']=True
mpl.rcParams['xtick.major.size']=7

mpl.rcParams['ytick.major.size']=7

mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['axes.edgecolor']='black'

mpl.rcParams['axes.edgecolor']='black'


mpl.rcParams['savefig.edgecolor']='black'
mpl.rcParams['savefig.facecolor']='white'


mpl.rcParams['axes.linewidth']=1

font = {'size'   : 13}

mpl.rc('font', **font)

mpl.rcParams['lines.linewidth']=3
mpl.rcParams['axes.labelsize']=16
mpl.rcParams['axes.grid']=False

import torch
from masking import *

import csv
import os
import json

from latencies import *
from utils.temchinData import data_temchin2008

from rbf import RBFNet

In [ ]:
fs=48828


#depends on expe:
I0 = 100 +10 - 32 #intensity ref for masker    #100 dB rms=1  +10 dB amp 5   (/sqrt(2))   #-32 dB masker atten
I0 = 10*np.log10( 10**(I0/10)/(fs/2) ) #spectral density

#Note for later: import common script to analysis instead?

### First estimation Q10 

In [ ]:
def lin_reg(x, y):
    x0=np.mean(x)
    y0=np.mean(y)
    x2=x-x0
    y2=y-y0
    
    a=np.mean(x2*y2)/np.mean(x2**2)
    
    b=y0-a*x0
    return a,b

In [ ]:

freqs_t=np.array(data_temchin2008[::2])
freqs_t=10**freqs_t
q10_t=np.array(data_temchin2008[1::2])



In [ ]:
bf_anf=[]
q10_anf=[]
with open('results/q101-22/q10s_clean.csv', 'r') as f:
    reader=csv.DictReader(f)
    for row in reader:
        bf_anf.append(float(row['CF'])*1e3)
        
        q10_anf.append(float(row['Q10']))
        

In [ ]:
freqs=np.array([1.5, 2.2, 3, 4, 5, 6, 8])*1e3
bw10=np.array([800, 2300, 600, 1000, 1300, 1400, 2600])
bw10_C=np.array([850,2400,850,1000,1250,2200,2500])


bw10_R=np.array([850,1450,350,1000,1250,950,2950])

pl.plot(freqs, freqs/bw10, '+', label='estimated w/ model')

pl.plot(freqs_t, q10_t, label='Temchin et al 2008')
pl.xlim([1000, 10000])
#pl.plot(freqs, freqs/bw10_C, '+', label='C')

#pl.plot(freqs, freqs/bw10_R, '+', label='R')

#pl.legend()

#lin reg
a_t, b_t = lin_reg(np.log(freqs_t), np.log(q10_t) )
a, b = lin_reg(np.log(freqs), np.log(freqs/bw10) )

f=np.linspace(1000, 10000)


#pl.plot(f, np.exp(b_t)*f**a_t)
#pl.plot(f, np.exp(b)*f**a)


pl.plot(bf_anf, q10_anf, 'x', label='ANF data')

pl.xscale('log')

pl.xlabel('f (Hz)')
pl.ylabel('Q10')
pl.legend(loc='lower right')
#pl.savefig('q10_estim.png')

pl.ylim([0.5, 7])
pl.show()

#pl.yscale('log')

### Estimations with optim

#### I/O funcs

Runs

In [ ]:
#expe_name='12-13' #'1-22'  #'4-23'  #'9-10' #'9-10_bis' #'12-13'

#results_name=''
#results_name='test-0720b'  #'test-0609'
#results_name= 'test-0914'
#results_name='test-0719b'


expe_name='1-22'

#results_name='012122-run0'
#results_name='012122-run1'  #distrib True I0_distrib False
#results_name='012122-run2'  #distrib True I0_distrib True
#results_name='012122-run3' #same with noise levels corrected
#results_name='012122-run4'  #alpha increased for E0
#results_name='012222-run5' #plus_lambda True, changed init 8khz
#results_name='013122-run6' #distrib True I0_distrib True new strategy init E0
results_name='013122-run7' 


#expe_name='4-23'

#results_name='012222-run0'
#results_name='012222-run1'  #distrib True I0_distrib False n_dim 7  #Q10 RBF erased
#results_name='012222-run2' #n_dim 4
#results_name='012222-run3'  #distrib True I0_distrib True
#results_name='012222-run4'  #distrib True I0_distrib True plus_lambda True

#expe_name='9-10'

#results_name='013122-run0'
#results_name='013122-run1' #distrib True I0_distrib False n_dim 7
#results_name='013122-run2' #distrib True I0_distrib True 
#results_name='013122-run3' #distrib True I0_distrib True plus_lambda True
#results_name='013122-run4' #distrib True I0_distrib True new strategy init E0
#results_name='013122-run5'
#results_name='013122-run6'
#results_name='013122-run7'

#expe_name='12-13'

#results_name='122721-run0
#results_name='122821-run1'  #distrib True #I0 distrib false
#results_name='010321-testerrs' #for testing purposes
#results_name='010321-run2'  #distrib True #I0 distrib True  #note: wrong year (2022)
#results_name='012022-run3'  #distrib True #I0 distrib True #plus_lambda True
#results_name='012022-run4'  #test higher n_dim (12) 

In [ ]:


if os.path.exists(f'./results/fit{expe_name}-{results_name}'):  #try to find parameters
    with open(f'./results/fit{expe_name}-{results_name}/distrib_params.json') as f:
        params = json.load(f)
        

        Q10_distributed=params['Q10_distributed']
        E0_distributed=params['E0_distributed']
        I0_distributed=params['I0_distributed']
        
        try:
            plus_lambda=params['plus_lambda']  #works with I0_distrib
        except KeyError as e:
            print("key 'plus_lambda' not found, considering False")
            plus_lambda=False
        
        CFs=params['CFs']
        print(f'Q10_distributed: {Q10_distributed} E0 distrib: {E0_distributed}, I0 distrib: {I0_distributed}\n CFs: {CFs}')
else:
    Q10_distributed=True
    E0_distributed=True #not used as results for E0 saved in same files either way
    I0_distributed=False
    CFs=[1500, 2200, 3000, 4000, 5000, 6000, 8000]

if results_name != '':
    results_folder=f'./results/fit{expe_name}-{results_name}/'
else:
    if Q10_distributed:
        results_folder=f'./results/fit{expe_name}-distrib/'
    else:
        results_folder=f'./results/fit{expe_name}/'
    
print(f'using as results folder: {results_folder}')
CF=5000

In [ ]:
wb_cdf=WeibullCDF_IOFunc.load_from_npz(f'{results_folder}/wbcfdIO_1st_estim_{CF}.npz')
wb_cdf2=WeibullCDF_IOFunc.load_from_npz(f'{results_folder}/wbcfdIO_{CF}.npz')

In [ ]:
with np.load(f'{results_folder}/maskamountCAP_{CF}.npz') as data:
    I_pts=data['I_pts']
    maskamount=data['maskamount']
    


I/O func

In [ ]:
I=np.linspace(-30, 30)


#pl.plot(I, sigm(torch.tensor(I)), label='fit sigm')
pl.plot(I, wb_cdf(torch.tensor(I)), label='fit wbcdf')

ma=wb_cdf2(torch.tensor(I))
if len(ma.shape)>1:
    ma=torch.squeeze(ma,0)
pl.plot(I, ma, label='after optim')
pl.xlabel('Power spectral density (dB)')

pl.plot(I_pts, maskamount, '+', markersize=10, markeredgewidth=3)

pl.plot(wb_cdf2._Iref, 1, '+', markersize=10, markeredgewidth=3, color='purple')

pl.ylabel('masking (max: broadband)')

pl.xlim(-20,20)
pl.ylim([0, 1.2])
pl.legend()

In [ ]:
I=np.linspace(-30, 30)


fig=pl.figure()
#ax=fig.add_axes([0,0,0.8,0.8])
ax=pl.gca()

ks=[]
I0s=[]
scales=[]
for CF in CFs:
    wb_cdf=WeibullCDF_IOFunc.load_from_npz(f'{results_folder}/wbcfdIO_{CF}.npz')
    ks.append(wb_cdf.k)
    I0s.append(wb_cdf.I0)
    scales.append(wb_cdf.scale)
    ma= wb_cdf(torch.tensor(I))
    if len(ma.shape)>1:
        ma=torch.squeeze(ma,0)
    CF_label = int(CF//1e3) if CF%1000 ==0 else f'{CF/1e3:.2f}'     
    pl.plot(I, ma*100, label=f'CF={CF_label} kHz')
    
pl.xlabel('Power spectral density (dB)')

pl.ylabel('Masking (%)')
pl.legend()

for key, spine in ax.spines.items():
    spine.set_visible(True)
    
    spine.set_linewidth(1.3)
    spine.set_edgecolor('black')



pl.axvline(wb_cdf._Iref, linestyle='--', c='lightgray')    
    

pl.axhline(100, linestyle='--', c='lightgray')

if expe_name=='9-10':
    pl.xlim([0, 30])
else:
    pl.xlim([-15, 20])
#pl.savefig('masking_io.svg')
pl.show()



fig=pl.figure()
ax=pl.gca()

p1, = pl.plot(CFs, ks, label='k', color='C0')

pl.ylabel('k')

ax.twinx()
if I0_distributed:
    
    with np.load(f'{results_folder}/I0_from_RBF.npz') as data:
        label = 'I0+scale RBF' if plus_lambda else 'I0 RBF'
        p2, = pl.plot(data['f'], data['I0'], label=label, color='C1')

else:
    p2, = pl.plot(CFs, np.array(I0s), label='I0', color='C1')
    
pl.ylabel('I0')

ax2=ax.twinx()
p3, = pl.plot(CFs, scales, label='scale', color='C2')
ax2.spines["right"].set_position(("axes", 1.2))

pl.xlabel('CF (Hz)')
pl.ylabel('scale')

lines = [p1, p2, p3]
ax.legend(lines, [l.get_label() for l in lines])

pl.xlim([CFs[0]-500, CFs[-1]+500])
pl.show()

In [ ]:
k_func = lambda f: np.interp(f, np.array(CFs), np.array(ks))
#k_func = lambda f: ks[-1]


scale_func = lambda f: np.interp(f, np.array(CFs), np.array(scales))


if I0_distributed:
    '''net_I0=RBFNet.create_from_jsonfile(f'{results_folder}/RBF_I0_params.json')
    with np.load(f'{results_folder}/I0_RBF_weights.npz') as data:
        rbf_weights=data['weights']
    net_I0.l2.weight.data=torch.tensor(rbf_weights)
    I0_func=net_I0
    ''' #issues with tensor format/numpy
    
    with np.load(f'{results_folder}/I0_from_RBF.npz') as data:
        RBF_f=data['f']
        RBF_I0=data['I0']
    if plus_lambda:
        I0_func = lambda f: (np.interp(f, RBF_f, RBF_I0) - scale_func(f))
    else:
        I0_func = lambda f: np.interp(f, RBF_f, RBF_I0)
    
else:
    I0_func = lambda f: np.interp(f, np.array(CFs), np.array(I0s))

#scale_func=lambda f: np.array(scales)[-1]
Iref=wb_cdf._Iref
constrained_at_Iref=wb_cdf.constrained_at_Iref

f_lin, level_lin, isomaskcurves=isomasking_curves_wbcdf(CFs[0]-500., CFs[-1]+1000., 20, 21, I0_func, scale_func, k_func,
            constrained_at_Iref=constrained_at_Iref, Iref=Iref, eps=0.001)



In [ ]:

fig=pl.figure()
#ax=fig.add_axes([0,0,0.8,0.8])

for i, tup in enumerate(zip( isomaskcurves, level_lin)):
    curve, level=tup
    if (i%4==0 or i == 2 or len(level_lin)-i==2) and i!=0:
        kargs= {'color':'black', 'linestyle':'--'} if level ==1 else {}
        pl.plot(f_lin*1e-3, curve, label=f'{level*100:.0f} % masking', **kargs)


for key, spine in ax.spines.items():
    spine.set_visible(True)
    
    spine.set_linewidth(1.3)
    spine.set_edgecolor('black')

    
#pl.ylabel()
pl.title('Isomasking curves')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
pl.xlabel('f (kHz)')
pl.ylabel('Power spectral density (dB)')
#pl.savefig('masking_iso.svg')
pl.show()

#### E0

In [ ]:

fig=pl.figure()
ax=fig.add_axes([0,0,0.6,0.6])
#ax=pl.gca()

for CF in CFs: #8000
    with np.load(f'{results_folder}/E0_{CF}.npz') as data:
        f=data['f']
        E0=data['E0']
        E0_amp=data['E0_amp']
        f=np.linspace(f[0], f[-1], len(E0)) #HACK err for now
        f_ind=(f<8500)*(f>1500)
        f_left_ind=f<1500
        f_right_ind=f>=8500
        p=pl.plot(f[f_ind]*1e-3, E0[f_ind], label=f"CF={CF} Hz") 
        
        pl.plot(f[f_left_ind]*1e-3, E0[f_left_ind], '--', color= p[0].get_color()) #label=f"CF={CF} Hz",
        pl.plot(f[f_right_ind]*1e-3, E0[f_right_ind], '--', color= p[0].get_color()) #label=f"CF={CF} Hz"
    if E0_distributed:
        break
        
pl.ylabel('weights R_0')
pl.xlabel('CF (kHz)')
pl.ylim(bottom=0)

#'theoretical' value? 
# Greenwood function, Eldredge et al 1981, Greenwood 1990
# anyway, very close to exponential value >1kHz
A=163.5
gamma=0.85
dx=1/(f+gamma*A) #up to prop. factor  

#pl.plot(f*1e-3, 40000*dx)

if not E0_distributed:
    pl.legend()

for key, spine in ax.spines.items():
    spine.set_visible(True)
    
    spine.set_linewidth(1.3)
    spine.set_edgecolor('black')
#pl.savefig('weights_R0.svg')


if not E0_distributed:
    pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


#pl.savefig('fweights.svg')
pl.show()

Latencies

In [ ]:
ax=None
for CF in CFs: #8000
    lat_model=PowerLawLatencies.load_from_npz(f'{results_folder}/lat_{CF}.npz')
    lat_model.name=f'CF {CF}'
    ax=plotLatencies(lat_model, ax=ax) #logscale=True
print(lat_model)

Equivalent E0 in time

$w(f) df = w(f) A \alpha \tau^{\alpha-1} d\tau$

In [ ]:

fig=pl.figure()
ax=fig.add_axes([0,0,0.6,0.6])

kargs={}

for CF in CFs: 
    with np.load(f'{results_folder}/E0_{CF}.npz') as data:
        f=data['f']
        E0=data['E0']
        E0_amp=data['E0_amp']
        f=np.linspace(f[0], f[-1], len(E0)) #HACK err for now
        f_ind=(f<8500)*(f>1500)
        f_left_ind=f<1500
        f_right_ind=f>=8500
        
    for ind in f_ind, f_left_ind, f_right_ind: 
        f_temp=f[ind]
        E0_temp=E0[ind]
        t=lat_model.t_from_f(f_temp).detach().numpy()
        A=lat_model.A.detach().numpy()
        alpha=lat_model.alpha.detach().numpy()
        t0=lat_model.t0.detach().numpy()
        w=E0_temp*A*alpha*np.power(np.abs(t-t0), alpha-1)
        w=np.abs(w)

        pl.plot(t*1e3, w*1e-8, **kargs)  #color=p[0].get_color()
        kargs={'linestyle':'--'}

        #pl.xlim([5.6, 7])
        
    if E0_distributed:
        break


for CF in CFs:
    t_CF=lat_model.t_from_f(CF).detach().numpy()
    ind=int((CF-f[0])/f[-1]*len(E0))
    w_CF=E0[ind]*A*alpha*np.power(np.abs(t_CF-t0), alpha-1)
    w_CF=np.abs(w_CF)
    dx=0.05
    dy=0.01*np.amax(w*1e-8)
    pl.arrow(t_CF*1e3+1.5*dx, w_CF*1e-8+1.5*dy, -dx, -dy, head_width=0.03, head_length=0.1,
             fc='k', ec='k', head_starts_at_zero=True)
    pl.text(t_CF*1e3+1.2*dx, w_CF*1e-8+1.2*dy, f'{CF} Hz')
pl.xlabel('t (ms)')


for key, spine in ax.spines.items():
    spine.set_visible(True)
    
    spine.set_linewidth(1.3)
    spine.set_edgecolor('black')

if expe_name == '12-13':
    pl.xlim([5, 8])
    pl.ylim([0,2])
    
if expe_name == '1-22':
    pl.xlim([5.7,7])
    pl.ylim([0,10])
    
if expe_name == '9-10':
    pl.xlim([5.8,7])
    pl.ylim([0,0.4])
        
if expe_name == '4-23':
    pl.xlim([5.9,7.5])
    pl.ylim([0,6])
    
#pl.savefig('fweights_t.svg')

#### Unitary responses

In [ ]:
fig=pl.figure()
ax=fig.add_axes([0,0,1,1])
for CF in CFs:
    with np.load(f'{results_folder}/ur_{CF}.npz') as data:
        ur=data['ur']
        t2=data['t2']
        pl.plot((t2-t2[0])*1e3, ur, label=f"CF={CF} Hz")
        pl.xlim([0,5])
pl.xlabel('t (ms)')
pl.title('Unitary responses')
pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')




for key, spine in ax.spines.items():
    spine.set_visible(True)
    
    spine.set_linewidth(1.3)
    spine.set_edgecolor('black')
    
pl.show()

corrected by E_amp

In [ ]:

for CF in CFs:
    with np.load(f'{results_folder}/ur_{CF}.npz') as data:
        ur=data['ur']
        t2=data['t2']

    with np.load(f'{results_folder}/E0_{CF}.npz') as data:
        #f=data['f']
        #E0=data['E0']
        E0_amp=data['E0_amp']
        
    pl.plot(t2*1e3, E0_amp*ur, label=f"CF={CF} Hz")
    pl.xlim([3,8])
pl.xlabel('t (ms)')
pl.title('Unitary responses (corrected w/ E_amp)')
pl.legend()
pl.show()

normalized re:max

In [ ]:

for CF in CFs:
    with np.load(f'{results_folder}/ur_{CF}.npz') as data:
        ur=data['ur']
        t2=data['t2']
        pl.plot(t2*1e3, ur/np.amax(np.abs(ur)), label=f"CF={CF} Hz")
        pl.xlim([3,8])
pl.xlabel('t (ms)')
pl.title('Unitary responses (normalized re:max)')
pl.legend()
#pl.savefig('ur.svg')
pl.show()

#### Q10

In [ ]:
if Q10_distributed:
    with np.load(f'{results_folder}/Q10.npz') as data:
        pl.plot(data['f'], data['Q10'], label=f'estim RBF {expe_name}')
        pl.ylabel('Q_10')
        pl.xlabel('f (Hz)')
        pl.xlim([1000, 9000])
        pl.ylim([0,8])

q10_gridsearch=[]
for CF in CFs:
    with np.load(f'{results_folder}/Q10gridsearch_{CF}.npz') as data:
        q10_gridsearch.append(CF/data['bw10_est'])
pl.plot(CFs, q10_gridsearch, '+', label=f'grid search {expe_name}')
        
#pl.plot(bf_anf, q10_anf, 'x', label='ANF data 1-22')

pl.plot(freqs_t, q10_t, label='Temchin et al 2008')
pl.legend()

Repeat grid search

*Note: for first runs, it does not correspond to mean error*

In [ ]:
CF=5000

with np.load(f'{results_folder}/Q10gridsearch_{CF}.npz') as data:
    bws=data['bw']
    errs=data['errs']
    pl.plot(bws, np.sqrt(errs))  #already in μV 
    pl.xlabel('Model 10-dB bandwidth (Hz)')

    pl.ylabel('RMS error (μV)')

####  Errors

In [ ]:

for CF in CFs:
    err= np.load(f'{results_folder}/err_list_{CF}.npy', allow_pickle=True)
    p = pl.plot( (1-err)*100, label=f'{CF} Hz')
    
    err_dic=np.load(f'{results_folder}/err_list_{CF}.npz')
    snr=err_dic['snr']
    snr=np.repeat(snr, len(err))
    pl.plot( (1-1/snr)*100, linestyle='--', color=p[0].get_color())
    
#pl.ylim([0.001, 0.006])
pl.xlabel('Steps')
pl.ylabel('% variance explained')
pl.xlim([0, 120])
pl.ylim([0, 100])
pl.legend()

Errors inside 100% region Tukey window

In [ ]:
plot_vfreq=False
df_pts=150  #120
if os.path.exists(f'{results_folder}/err_list_{CF}_inside_window.npz'):
    for CF in CFs:
        err_dic=np.load(f'{results_folder}/err_list_{CF}_inside_window.npz')
        
        if CF==CFs[0]:
            kwargs=[{'label': label} for label in ['signal RMS overall', 'signal RMS various notch atten', 
                                               'signal RMS various notch width', 'prediction err overall', 'err various notch atten', 
                                               'err RMS various notch width', 'estimated noise level', 'sig RMS vfreq', 'err RMS vfreq']]
        else:
            kwargs=[{} for label in ['signal RMS overall', 'signal RMS various notch atten', 
                                               'signal various RMS notch width', 'prediction err overall', 'err various notch atten', 
                                               'err various RMS notch width', 'estimated noise level', 'sig RMS vfreq', 'err RMS vfreq'] ]
        pl.scatter(CF, err_dic['sig_rms']*1e3, color='C0', marker='o', **kwargs[0])
        pl.scatter(CF+df_pts, err_dic['sig_rms_ntch']*1e3, color='C0', marker='+', **kwargs[1])
        pl.scatter(CF+2*df_pts, err_dic['sig_rms_vbw']*1e3, color='C0', marker='d', **kwargs[2])
        
        pl.scatter(CF, err_dic['err_rms']*1e3, color='C1', marker='o', **kwargs[3])
        pl.scatter(CF+df_pts, err_dic['err_rms_ntch']*1e3, color='C1', marker='+', **kwargs[4])
        pl.scatter(CF+2*df_pts, err_dic['err_rms_vbw']*1e3, color='C1', marker='d', **kwargs[5])
    
        pl.scatter(CF, err_dic['noise_rms']*1e3, color='C2', marker='o', **kwargs[6])
        
        if plot_vfreq:
            try:
                pl.scatter(CF+3*df_pts, err_dic['sig_rms_vfreq']*1e3, color='C0', marker='X', **kwargs[7])
                pl.scatter(CF+3*df_pts, err_dic['err_rms_vfreq']*1e3, color='C1', marker='X', **kwargs[8])
            except KeyError as e:
                pass
    pl.xlabel('CF (kHz)')
    pl.ylabel('RMS value (μV)')
    
    pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    #pl.savefig('err_Tukey.svg')

variance explained inside Tuckey window

In [ ]:
if os.path.exists(f'{results_folder}/err_list_{CF}_inside_window.npz'):
    for CF in CFs:
        err_dic=np.load(f'{results_folder}/err_list_{CF}_inside_window.npz')
        
        if CF==CFs[0]:
            kwargs=[{'label': label} for label in ['variance explained overall', 'variance explained notch atten', 
                                               'variance explained various notch width', 
                                                  'variance when removing noise variance overall',  'variance when removing noise variance notch atten', 
                                                'variance when removing noise variance various notch width'
                                                  ]]
        else:
            kwargs=[{} for label in ['signal RMS overall', 'signal RMS various notch atten', 
                                               'signal various RMS notch width', 'prediction err overall', 'err various notch atten', 
                                               'err various RMS notch width', 'estimated noise level', 'sig RMS vfreq', 'err RMS vfreq'] ]
        pl.scatter(CF, (1-err_dic['err_rms']**2/err_dic['sig_rms']**2)*100, color='C0', marker='o', **kwargs[0])
        pl.scatter(CF+100, (1-err_dic['err_rms_ntch']**2/err_dic['sig_rms_ntch']**2)*100, color='C0', marker='+', **kwargs[1])
        pl.scatter(CF+200, (1-err_dic['err_rms_vbw']**2/err_dic['sig_rms_vbw']**2)*100, color='C0', marker='d', **kwargs[2])
        
        pl.scatter(CF,(1-err_dic['noise_rms']**2/err_dic['sig_rms']**2)*100, color='C1', marker='o', **kwargs[3])
        pl.scatter(CF+100,(1-err_dic['noise_rms']**2/err_dic['sig_rms_ntch']**2)*100, color='C1', marker='+', **kwargs[4])
        pl.scatter(CF+200, (1-err_dic['noise_rms']**2/err_dic['sig_rms_vbw']**2)*100, color='C1', marker='d', **kwargs[5])
    
        
        '''try:
            pl.scatter(CF+300, err_dic['sig_rms_vfreq']*1e3, color='C0', marker='X', **kwargs[7])
            pl.scatter(CF+300, err_dic['err_rms_vfreq']*1e3, color='C1', marker='X', **kwargs[8])
        except KeyError as e:
            pass
        '''
    pl.xlabel('CF (kHz)')
    pl.ylabel('Variance explained (%)')
    
    pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')


### Results multiple-expe

In [ ]:
expe_names=['1-22', '4-23', '9-10', '12-13']  #'1-22'  #'4-23'  #'9-10'


# '9-10_bis'  -  'test-0920b'
#results_name=''
#results_names=['test-0720b', 'test-0719b', 'test-0914', '012022-run3']  #'test-0609'  #old runs

results_names=['013122-run7', '012222-run4', '013122-run7', '012022-run3']  #'test-0609'



dics_params=[] #stores params

for expe_name2, results_name in zip(expe_names, results_names):
    if os.path.exists(f'./results/fit{expe_name2}-{results_name}'):  #try to find parameters
        with open(f'./results/fit{expe_name2}-{results_name}/distrib_params.json') as f:
            params = json.load(f)

            Q10_distributed=True
            E0_distributed=params['E0_distributed']
            I0_distributed=params['I0_distributed']
            CFs=params['CFs']
            print(f'{expe_name2}, E0 distrib :{E0_distributed}, I0 distrib: {I0_distributed}\n CFs: {CFs}')
    else:
        Q10_distributed=True
        E0_distributed=True #not used as results for E0 saved in same files either way
        I0_distributed=False
        CFs=[1500, 2200, 3000, 4000, 5000, 6000, 8000]

    if results_name != '':
        results_folder=f'./results/fit{expe_name2}-{results_name}/'
    else:
        if Q10_distributed:
            results_folder=f'./results/fit{expe_name2}-distrib/'
        else:
            results_folder=f'./results/fit{expe_name2}/'
    
    dic_params={"Q10_distributed":Q10_distributed, "E0_distributed":E0_distributed, 
                "I0_distributed":I0_distributed, "CFs":CFs, "results_folder":results_folder}
    dics_params.append(dic_params)
#print(f'using as results folder: {results_folder}')
CF=6000

#### Q10

In [ ]:
arr_data=[]
colors_arr=['#2393caff', '#f09168ff', '#e5ae38ff', '#5da668ff']
for expe_name2, results_name, dic_params, color_p in zip(expe_names, results_names, dics_params, colors_arr):
    results_folder=dic_params["results_folder"]
    Q10_distributed=dic_params["Q10_distributed"]    
    CFs=dic_params["CFs"]
    
    
    with np.load(f'{results_folder}/Q10.npz') as data:
        data_f=data['f']
        arr_data.append(data['Q10'])
        p=pl.gca().plot(data['f']*1e-3, data['Q10'], label=f'estim RBF {expe_name2}', linewidth=2, color=color_p)
        pl.ylabel('Q_10')
        pl.xlabel('f (kHz)')
        pl.xlim([1, 9])
        pl.ylim([0.,6])

    q10_gridsearch=[]
    for CF in CFs:
        with np.load(f'{results_folder}/Q10gridsearch_{CF}.npz') as data:
            q10_gridsearch.append(CF/data['bw10_est'])
    pl.plot(np.array(CFs)*1e-3, q10_gridsearch, '+', label=f'grid search {expe_name2}', color=p[0].get_color())

    #pl.plot(bf_anf, q10_anf, 'x', label='ANF data 1-22')

    
arr_data=np.stack(arr_data)

arr_std=arr_data.std(axis=0)

arr_err=arr_std/np.sqrt(len(expe_names))

arr_avg=np.mean(arr_data, axis=0)

p, = pl.plot(data_f*1e-3, arr_avg, label='average', linewidth=3, color='C4') 


pl.plot(data_f*1e-3, arr_avg-arr_std, alpha=0.9, color=p.get_color(), linewidth=1)
pl.plot(data_f*1e-3, arr_avg+arr_std, alpha=0.9, color=p.get_color(), linewidth=1)
ax=pl.gca()
ax.fill_between(data_f*1e-3, arr_avg-arr_std, arr_avg+arr_std, alpha=0.3, color=p.get_color())
    
pl.plot(freqs_t*1e-3, q10_t, label='Temchin et al 2008', linewidth=4, color='C5')

pl.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

#pl.savefig('estim_Q10.svg')

#pl.xscale('log')
#pl.yscale('log')